# 2D Histogram and Kalman Filters

- 2D Histogram Filters
- Kalman Filters
- Representing state & motion

![Craven](https://static01.nyt.com/images/2015/02/19/us/19craven-obit/19craven-obit-facebookJumbo.jpg)

John Craven

A prori

![priori](images/priori.jpg)

Update

![up](images/update.jpg)

A posteriori

![post](images/posteriori.jpg)

## Kalman Filter

A tool to **combine information** in the presence of **uncertainty**

It is a two-stage filter:
1. Prediction
2. Correction (Update)

In a simple case our **state** contains only the position and velocity of our robot:

$$
\vec{x} = \begin{bmatrix}
p\\
v
\end{bmatrix}$$

Given uncertain or inaccurate information we might end up with an undesired result 

![Fall](https://www.bzarg.com/wp-content/uploads/2015/08/robot_ohnoes.png)

Possible combinations of position and velocity:

![PV](https://www.bzarg.com/wp-content/uploads/2015/08/gauss_0-287x300.png)
![PV2](https://www.bzarg.com/wp-content/uploads/2015/08/gauss_1-300x267.png)

Source: https://www.bzarg.com

We don't know the actual position, but we understand that some combinations of position and velocity are more likely than others.

Both variables are assumed to be **random and with a Gaussian distirbution**. Each variable has:
- a **mean** corresponding to the most likely state
- a **variance** corresponding to the uncertainty


![Correlated](https://www.bzarg.com/wp-content/uploads/2015/08/gauss_2-300x276.png)

So know we have an estimate of our state vector and we have the covariance matrix:

$$
\begin{equation} \label{eq:statevars}
\begin{aligned}
\mathbf{\hat{x}}_k &= \begin{bmatrix}
\text{position}\\
\text{velocity}
\end{bmatrix}\\
\mathbf{P}_k &=
\begin{bmatrix}
\Sigma_{pp} & \Sigma_{pv} \\
\Sigma_{vp} & \Sigma_{vv} \\
\end{bmatrix}
\end{aligned}
\end{equation}
$$

### Step 1: Prediction

![Prediction](https://www.bzarg.com/wp-content/uploads/2015/08/gauss_7-300x276.jpg)
![KF](images/kf.jpg)

Source: Self-Driving Car Specialization Resources on Coursera by University of Toronto

![Matrix prediction](https://www.bzarg.com/wp-content/uploads/2015/08/gauss_8-300x276.jpg)

So, we have our previous state at time *k-1*:

$$
\begin{equation} \label{eq:statevars}
\begin{aligned}
\color{royalblue}{\mathbf{\hat{x}}_{k-1}} &=
\begin{bmatrix}
p_{k-1} \\
v_{k-1} \\
\end{bmatrix}
\end{aligned}
\end{equation}
$$

Now we want to predict our state at time *k*. For this we can first think about the underlying formulas which describe what we know about how the car should move:

$$
\begin{split}
{p_k} &= \color{royalblue}{p_{k-1}} + \Delta t &\color{royalblue}{v_{k-1}} \\
{v_k} &= &\color{royalblue}{v_{k-1}}
\end{split}
$$ 


Then we can see this formulas as a matrix multiplication: 

$$
\begin{align}
{\mathbf{\hat{x}}_k} &= \begin{bmatrix}
1 & \Delta t \\
0 & 1
\end{bmatrix} \color{royalblue}{\mathbf{\hat{x}}_{k-1}} \\
&= \mathbf{F}_k \color{royalblue}{\mathbf{\hat{x}}_{k-1}} \label{statevars}
\end{align}
$$

To avoid confusion. Sigma is often used to mean "sum", but it is also often used as the name for a covariance matrix. 

$$
\begin{equation}
\begin{split}
Cov(x) &= \Sigma\\
Cov(\color{firebrick}{\mathbf{F}}x) &= \color{firebrick}{\mathbf{F}} \Sigma \color{firebrick}{\mathbf{F}}^T
\end{split} \label{covident}
\end{equation}
$$

Applied to our case we get:

$$
\begin{equation}
\begin{split}
{\mathbf{\hat{x}}_k} &= \mathbf{F}_k \color{royalblue}{\mathbf{\hat{x}}_{k-1}} \\
{\mathbf{P}_k} &= \mathbf{F_k} \color{royalblue}{\mathbf{P}_{k-1}} \mathbf{F}_k^T
\end{split}
\end{equation}
$$

**Additional information by known external forces**

Imagine that we also can know the expected acceleration of our car:

$$
\begin{split}
\color{deeppink}{p_k} &= \color{royalblue}{p_{k-1}} + {\Delta t} &\color{royalblue}{v_{k-1}} + &\frac{1}{2} \color{darkorange}{a} {\Delta t}^2 \\
\color{deeppink}{v_k} &= &\color{royalblue}{v_{k-1}} + & \color{darkorange}{a} {\Delta t}
\end{split}
$$ In matrix form: $$
\begin{equation}
\begin{split}
\color{deeppink}{\mathbf{\hat{x}}_k} &= \mathbf{F}_k \color{royalblue}{\mathbf{\hat{x}}_{k-1}} + \begin{bmatrix}
\frac{\Delta t^2}{2} \\
\Delta t
\end{bmatrix} \color{darkorange}{a} \\
&= \mathbf{F}_k \color{royalblue}{\mathbf{\hat{x}}_{k-1}} + \mathbf{B}_k \color{darkorange}{\vec{\mathbf{u}_k}}
\end{split}
\end{equation}
$$

Here 
$$\mathbf{B}_k $$ is called the control matrix and $$\color{darkorange}{\vec{\mathbf{u}_k}} $$ the control vector.

**Additional information by unknown external forces**

![External unknown](https://www.bzarg.com/wp-content/uploads/2015/08/gauss_10a.jpg)

To take this into account we can model these unknown external forces by adding noise **Q** to our covariance:

$$
\begin{equation}
\begin{split}
\color{deeppink}{\mathbf{\hat{x}}_k} &= \mathbf{F}_k \color{royalblue}{\mathbf{\hat{x}}_{k-1}} + \mathbf{B}_k \color{darkorange}{\vec{\mathbf{u}_k}} \\
\color{deeppink}{\mathbf{P}_k} &= \mathbf{F_k} \color{royalblue}{\mathbf{P}_{k-1}} \mathbf{F}_k^T + \color{mediumaquamarine}{\mathbf{Q}_k}
\end{split}
\label{kalpredictfull}
\end{equation}
$$

![Q](https://www.bzarg.com/wp-content/uploads/2015/08/gauss_10b-300x300.jpg)

**Final result:** 
- The **new best estimate** is a prediction made from the previous best estimate, plus a correction for known external influences.

- The **new uncertainty** is predicted from the old uncertainty, with some additional uncertainty from the environment.

### Step 2: Correction

Applying our measurements to our prediction is similar to calculating the prediction given the previous state:

![Measurements](https://www.bzarg.com/wp-content/uploads/2015/08/gauss_13.jpg)
![Uncertainty](https://www.bzarg.com/wp-content/uploads/2015/08/gauss_14.jpg)

Now we have the Gaussian distribution corresponding to our prediction and the Gaussian distribution corresponding to the sensor measurements:

![Two](https://www.bzarg.com/wp-content/uploads/2015/08/gauss_4-300x276.jpg)

$$
\begin{equation}
\begin{aligned}
\vec{\mu}_{\text{expected}} = \mathbf{H}_k \color{deeppink}{\mathbf{\hat{x}}_k} \\
\mathbf{\Sigma}_{\text{expected}} = \mathbf{H}_k \color{deeppink}{\mathbf{P}_k} \mathbf{H}_k^T
\end{aligned}
\end{equation}
$$

Multiplying them together gives us the overlap:

![Overlap](https://www.bzarg.com/wp-content/uploads/2015/08/gauss_6-300x276.png)

How does it work and where do these come from?:

![Form](images/equations.jpg)

Source: Self-Driving Car Specialization by Toronto University

Let's take a look at how we can multiply two one-dimensional Gaussian curves together:

![1DGauss](https://www.bzarg.com/wp-content/uploads/2015/08/gauss_joint.png)

Normal distribution formula:

$$
\begin{equation} \label{gaussformula}
\mathcal{N}(x, \mu,\sigma) = \frac{1}{ \sigma \sqrt{ 2\pi } } e^{ -\frac{ (x \mu)^2 }{ 2\sigma^2 } }
\end{equation}
$$

We multiply both curves:

$$\begin{equation} \label{gaussequiv}
\mathcal{N}(x, \color{fuchsia}{\mu_0}, \color{deeppink}{\sigma_0}) \cdot \mathcal{N}(x, \color{yellowgreen}{\mu_1}, \color{mediumaquamarine}{\sigma_1}) \stackrel{?}{=} \mathcal{N}(x, \color{royalblue}{\mu}, \color{mediumblue}{\sigma})
\end{equation}$$

After plugging in the normal distribution equation into this equality we can apply algebra and re-write it like this:

$$
\begin{equation} \label{fusionformula}
\begin{aligned}
\color{royalblue}{\mu} = \mu_0 + \frac{\sigma_0^2 (\mu_1 - \mu_0)} {\sigma_0^2 + \sigma_1^2}\\
\color{mediumblue}{\sigma}^2 = \sigma_0^2 - \frac{\sigma_0^4} {\sigma_0^2 + \sigma_1^2}
\end{aligned}
\end{equation}$$

In both equations this part is repeated:

$$
\begin{equation} \label{gainformula}
\color{purple}{\mathbf{k}} = \frac{\sigma_0^2}{\sigma_0^2 + \sigma_1^2}
\end{equation} $$

Now we can simplify our equations by substituting this repeated part by the variable *k*:

$$
\begin{equation}
\begin{split}
\color{royalblue}{\mu} = \mu_0 + \color{purple}{\mathbf{k}} (\mu_1 - \mu_0)\\
\color{mediumblue}{\sigma}^2 = \sigma_0^2 - \color{purple}{\mathbf{k}} \sigma_0^2
\end{split} \label{update}
\end{equation} $$

And finally we re-write this into matrix form to generalize for more dimensions:

$$
\begin{equation} \label{matrixgain}
\color{purple}{\mathbf{K}} = \Sigma_0 (\Sigma_0 + \Sigma_1)^{-1}
\end{equation} $$ 

$$
\begin{equation}
\begin{split}
\color{royalblue}{\vec{\mu}} = \vec{\mu_0} + \color{purple}{\mathbf{K}} (\vec{\mu_1} - \vec{\mu_0})\\
\color{mediumblue}{\Sigma} = \Sigma_0 - \color{purple}{\mathbf{K}} \Sigma_0
\end{split} \label{matrixupdate}
\end{equation}
$$

Remember that our two Gaussian estimates were:

$$(\color{fuchsia}{\mu_0}, \color{deeppink}{\Sigma_0}) = (\color{fuchsia}{\mathbf{H}_k \mathbf{\hat{x}}_k}, \color{deeppink}{\mathbf{H}_k \mathbf{P}_k \mathbf{H}_k^T})$$

And $$ (\color{yellowgreen}{\mu_1}, \color{mediumaquamarine}{\Sigma_1}) = (\color{yellowgreen}{\vec{\mathbf{z}_k}}, \color{mediumaquamarine}{\mathbf{R}_k})$$

$$
\begin{equation}
\begin{aligned}
\mathbf{H}_k \color{royalblue}{\mathbf{\hat{x}}_k'} = \color{fuchsia}{\mathbf{H}_k \mathbf{\hat{x}}_k} + \color{purple}{\mathbf{K}} ( \color{yellowgreen}{\vec{\mathbf{z}_k}} - \color{fuchsia}{\mathbf{H}_k \mathbf{\hat{x}}_k} ) \\
\mathbf{H}_k \color{royalblue}{\mathbf{P}_k'} \mathbf{H}_k^T = \color{deeppink}{\mathbf{H}_k \mathbf{P}_k \mathbf{H}_k^T} \color{purple} -{\mathbf{K}} \color{deeppink}{\mathbf{H}_k \mathbf{P}_k \mathbf{H}_k^T}
\end{aligned} \label {kalunsimplified}
\end{equation}
$$

$$
\begin{equation} \label{eq:kalgainunsimplified}
\color{purple}{\mathbf{K}} = \color{deeppink}{\mathbf{H}_k \mathbf{P}_k \mathbf{H}_k^T} ( \color{deeppink}{\mathbf{H}_k \mathbf{P}_k \mathbf{H}_k^T} + \color{mediumaquamarine}{\mathbf{R}_k})^{-1}
\end{equation}
$$

We can simplify the equations by taking away Hk and the transpose from both sides of the equality 

$$
\begin{equation}
\begin{split}
\color{royalblue}{\mathbf{\hat{x}}_k'} = \color{fuchsia}{\mathbf{\hat{x}}_k} + \color{purple}{\mathbf{K'}} ( \color{yellowgreen}{\vec{\mathbf{z}_k}} - \color{fuchsia}{\mathbf{H}_k \mathbf{\hat{x}}_k} ) \\
\color{royalblue}{\mathbf{P}_k'} = \color{deeppink}{\mathbf{P}_k} - \color{purple}{\mathbf{K'}} \color{deeppink}{\mathbf{H}_k \mathbf{P}_k}
\end{split}
\label{kalupdatefull}
\end{equation} $$ 

$$
\begin{equation}
\color{purple}{\mathbf{K'}} = \color{deeppink}{\mathbf{P}_k \mathbf{H}_k^T} ( \color{deeppink}{\mathbf{H}_k \mathbf{P}_k \mathbf{H}_k^T} + \color{mediumaquamarine}{\mathbf{R}_k})^{-1}
\label{kalgainfull}
\end{equation}
$$

More resources on the Kalman Filter:
- https://www.bzarg.com/p/how-a-kalman-filter-works-in-pictures/